# 00 — Environment check and smoke/consistency tests (Windows-friendly)

This notebook validates your environment (hardware, drivers, libraries, providers), runs quick smoke and optional consistency tests, and generates reusable diagnostics artifacts.

It does not abort if something is missing; it records status and continues. Outputs are written under `reports/` for later use by other notebooks.

Artifacts:
- reports/env_report.json, reports/env_report.md
- reports/providers_status.csv
- reports/consistency_quickcheck.json (if models exist)
- reports/00_status.json


In [1]:
import os
from pathlib import Path

# Section B1 — Run environment collection and write env_report.json/md
from utils.env_report import gather_env_info, save_env_report

# Detect project root (assume this notebook is in notebooks/)
ROOT = Path(__file__).resolve().parent.parent if '__file__' in globals() else Path(os.getcwd()).parent
REPORTS = ROOT / 'reports'
METRICS = ROOT / 'metrics'
ASSETS = ROOT / 'tests' / 'assets'
ONNX_DIR = ROOT / 'models_saved' / 'onnx'
PT_DIR = ROOT / 'models_saved' / 'pytorch'
OV_DIR = ROOT / 'models_saved' / 'openvino_ir'

REPORTS.mkdir(parents=True, exist_ok=True)
METRICS.mkdir(parents=True, exist_ok=True)
ASSETS.mkdir(parents=True, exist_ok=True)
ONNX_DIR.mkdir(parents=True, exist_ok=True)
PT_DIR.mkdir(parents=True, exist_ok=True)
OV_DIR.mkdir(parents=True, exist_ok=True)

info = gather_env_info(ROOT)
save_env_report(info, str(REPORTS / 'env_report.json'), str(REPORTS / 'env_report.md'))
print('Saved reports/env_report.json and reports/env_report.md')
print('Providers:', info.get('onnxruntime', {}).get('available'))
print('GPU(s):', info.get('nvidia'))
print('OpenVINO devices:', info.get('openvino', {}).get('available'))
print('Write permissions:', info.get('write_permissions'))


Saved reports/env_report.json and reports/env_report.md
Providers: ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
GPU(s): {'gpu_name': 'NVIDIA GeForce RTX 4080 Laptop GPU', 'driver_version': '580.97'}
OpenVINO devices: ['CPU', 'GPU', 'NPU']
Write permissions: {'metrics': True, 'reports': True}


In [2]:
# Section C1 — PyTorch, ONNX, OpenVINO availability and basic checks
from utils.env_report import check_pytorch, check_onnxruntime, check_openvino

print('PyTorch:', check_pytorch())
print('ONNX Runtime:', check_onnxruntime())
print('OpenVINO:', check_openvino())


PyTorch: {'installed': True, 'version': '2.8.0+cu129', 'cuda_available': True, 'cuda_device_count': 1, 'cudnn': 91002}
ONNX Runtime: {'available': ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'], 'session_cpu': True, 'session_cuda': True, 'error': None}
OpenVINO: {'available': ['CPU', 'GPU', 'NPU'], 'error': None}


In [3]:
# Section C2 — Check write permissions for all relevant folders
from utils.env_report import check_write_permissions

folders = [REPORTS, METRICS, ASSETS, ONNX_DIR, PT_DIR, OV_DIR]
perms = {str(f): check_write_permissions(f) for f in folders}
print('Write permissions:', perms)


Write permissions: {'C:\\Users\\padul\\OneDrive\\Universidad\\Doctorado\\Desarrollo\\federated-lab-multihw\\reports': True, 'C:\\Users\\padul\\OneDrive\\Universidad\\Doctorado\\Desarrollo\\federated-lab-multihw\\metrics': True, 'C:\\Users\\padul\\OneDrive\\Universidad\\Doctorado\\Desarrollo\\federated-lab-multihw\\tests\\assets': True, 'C:\\Users\\padul\\OneDrive\\Universidad\\Doctorado\\Desarrollo\\federated-lab-multihw\\models_saved\\onnx': True, 'C:\\Users\\padul\\OneDrive\\Universidad\\Doctorado\\Desarrollo\\federated-lab-multihw\\models_saved\\pytorch': True, 'C:\\Users\\padul\\OneDrive\\Universidad\\Doctorado\\Desarrollo\\federated-lab-multihw\\models_saved\\openvino_ir': True}


## Section D — Smoke tests
We'll run a tiny end-to-end inference with ORT (CPU and optional CUDA) and OpenVINO CPU.


In [4]:
# Section D1 — Run basic model import and ONNX export checks
from utils.env_report import test_model_import_and_export

results = test_model_import_and_export(ONNX_DIR)
print('Model import/export results:', results)


Model import/export results: {'ok': True, 'onnx_path': 'C:\\Users\\padul\\OneDrive\\Universidad\\Doctorado\\Desarrollo\\federated-lab-multihw\\models_saved\\onnx\\tiny_envcheck.onnx', 'sha256': '0103917f5f74cb7c38269a77d640959e71f4004fd3b2725eb5d69721eff87461'}


In [5]:
# Section D2 — Run basic inference and consistency checks
from utils.env_report import test_inference_consistency

consistency = test_inference_consistency(ONNX_DIR)
print('Inference consistency:', consistency)


Inference consistency: {'ok': True, 'cpu_ok': True, 'cuda_ok': True, 'consistent': True, 'cpu_error': None, 'cuda_error': None}


In [6]:
# Section F — Global Pass/Fail summary and 00_status.json
from pathlib import Path
import json, csv

status = {
    'providers_ok': False,
    'gpu_cuda_ok': False,
    'write_permissions_ok': False,
    'npu_detected': False,
    'consistency_ok': None,
}

# Load env report for write permissions and OV devices
try:
    env = json.loads(Path(f'{REPORTS}/env_report.json').read_text(encoding='utf-8'))
    status['write_permissions_ok'] = bool(
        env.get('write_permissions', {}).get('reports') and env.get('write_permissions', {}).get('metrics'))
    ov_devs = env.get('openvino', {}).get('available', [])
    status['npu_detected'] = 'NPU' in (ov_devs or [])
except Exception:
    pass

# Providers status from CSV
try:
    rows = list(csv.reader(open(f'{REPORTS}/providers_status.csv', newline='', encoding='utf-8')))
    hdr = rows[0];
    data = rows[1:]
    idx = {name: i for i, name in enumerate(hdr)}


    def find(backend, prov):
        for r in data:
            if r[idx['backend']] == backend and r[idx['provider_or_device']] == prov:
                return r
        return None


    ort_cpu = find('ORT', 'CPU')
    ov_cpu = find('OpenVINO', 'CPU')
    status['providers_ok'] = bool(
        ort_cpu and ov_cpu and (ort_cpu[idx['test_infer_ok']] == 'True' or ort_cpu[idx['test_infer_ok']] == True) and (
                ov_cpu[idx['test_infer_ok']] == 'True' or ov_cpu[idx['test_infer_ok']] == True))
    ort_cuda = find('ORT', 'CUDA')
    if ort_cuda:
        status['gpu_cuda_ok'] = bool(ort_cuda[idx['test_infer_ok']] == 'True' or ort_cuda[idx['test_infer_ok']] == True)
except Exception:
    pass

# Consistency quickcheck
cc_path = Path(f'{REPORTS}/consistency_quickcheck.json')
if cc_path.exists():
    try:
        cc = json.loads(cc_path.read_text(encoding='utf-8'))
        status['consistency_ok'] = bool(cc.get('consistency_ok'))
    except Exception:
        status['consistency_ok'] = False

Path(f'{REPORTS}/00_status.json').write_text(json.dumps(status, indent=2), encoding='utf-8')

# Render human summary
mk = []
mk.append('## Summary\n')
mk.append(f"- Providers (ORT CPU + OV CPU): {'✅' if status['providers_ok'] else '❌'}\n")
mk.append(f"- GPU CUDA (if present): {'✅' if status['gpu_cuda_ok'] else '❌'}\n")
mk.append(f"- Write permissions (reports + metrics): {'✅' if status['write_permissions_ok'] else '❌'}\n")
mk.append(f"- NPU detected: {'✅' if status['npu_detected'] else '❌'}\n")
if status['consistency_ok'] is not None:
    mk.append(f"- Consistency quickcheck: {'✅' if status['consistency_ok'] else '❌'}\n")
print(''.join(mk))


## Summary
- Providers (ORT CPU + OV CPU): ✅
- GPU CUDA (if present): ✅
- Write permissions (reports + metrics): ✅
- NPU detected: ✅
- Consistency quickcheck: ✅

